# Lunar Survival Challenge Multi-Agent System Implementation
Implemented a comprehensive collaborative agent-based system for the lunar survival challenge. The system follows our redesigned workflow with paired agent teams and a focus on deeper collaboration.

In [ ]:
import os
import json
from typing import List, Dict, Any, Optional, Tuple
from dotenv import load_dotenv
from openai import OpenAI

# Load environment variables
load_dotenv()

class Agent:
    """Base agent class for lunar survival challenge."""
    
    def __init__(self, 
                 instruction: str, 
                 role: str, 
                 expertise_description: str,
                 model_info: str = 'gpt-4o-mini',
                 examples: Optional[List[Dict[str, str]]] = None):
        """
        Initialize an LLM-based agent with a specific role.
        
        Args:
            instruction: The system instruction for the agent
            role: The role of the agent (e.g., "Science Analyst")
            expertise_description: Description of the agent's expertise
            model_info: The LLM model to use
            examples: Optional examples to include in the prompt
        """
        self.instruction = instruction
        self.role = role
        self.expertise_description = expertise_description
        self.model_info = model_info
        self.examples = examples or []
        self.conversation_history = []
        
        # Initialize OpenAI client
        self.client = OpenAI(api_key=os.environ.get('OPENAI_API_KEY'))
        
        # Build initial system message
        self.messages = [
            {"role": "system", "content": self._build_system_prompt()}
        ]
        
        # Add example conversations if provided
        if self.examples:
            for example in self.examples:
                self.messages.append({"role": "user", "content": example['question']})
                self.messages.append({
                    "role": "assistant", 
                    "content": example['answer'] + "\n\n" + example.get('reason', '')
                })
                
    ###########################################################################
    
    def _build_system_prompt(self) -> str:
        """Build the system prompt for the agent."""
        prompt = f"You are a {self.role} who {self.expertise_description}. {self.instruction}"
        return prompt
    
    
    ###########################################################################
    
    def chat(self, message: str) -> str:
        """
        Send a message to the agent and get a response.
        
        Args:
            message: The message to send to the agent
            
        Returns:
            The agent's response
        """
        # Add the user message to the conversation
        self.messages.append({"role": "user", "content": message})
        
        # Get response from OpenAI
        response = self.client.chat.completions.create(
            model=self.model_info,
            messages=self.messages,
            temperature=0.7,
            max_tokens=1500
        )
        
        # Extract and store the response
        assistant_message = response.choices[0].message.content
        self.messages.append({"role": "assistant", "content": assistant_message})
        self.conversation_history.append({"user": message, "assistant": assistant_message})
        
        return assistant_message
    
    ###########################################################################
    
    def get_conversation_history(self) -> List[Dict[str, str]]:
        """Get the conversation history."""
        return self.conversation_history
    
    
######################################################################################################################################################
######################################################################################################################################################
######################################################################################################################################################


class Team:
    """Represents a collaborative team of agents working together."""
    
    def __init__(self, 
                 name: str, 
                 goal: str, 
                 members: List[Dict[str, str]],
                 examples: Optional[List[Dict[str, str]]] = None):
        """
        Initialize a team of agents.
        
        Args:
            name: Name of the team
            goal: The shared goal of the team
            members: List of member information (role, expertise_description)
            examples: Optional examples to include in agent prompts
        """
        self.name = name
        self.goal = goal
        self.examples = examples
        self.agents = []
        self.team_conversation = []
        
        # Create agents based on member information
        for member_info in members:
            agent = Agent(
                instruction=f"You are part of the {name} team. Your goal is to {goal}.",
                role=member_info['role'],
                expertise_description=member_info['expertise_description'],
                model_info='gpt-4o-mini',
                examples=examples
            )
            self.agents.append(agent)
            
    ###########################################################################
    
    def collaborate(self, topic: str, rounds: int = 3) -> List[Dict[str, str]]:
        """
        Facilitate collaboration between team members.
        
        Args:
            topic: The topic for collaboration
            rounds: Number of discussion rounds
            
        Returns:
            Conversation records
        """
        conversation_records = []
        
        # Initial prompt to each agent
        initial_responses = []
        for agent in self.agents:
            response = agent.chat(
                f"As a {agent.role}, please provide your initial thoughts on: {topic}"
            )
            initial_responses.append({
                "role": agent.role,
                "response": response
            })
            conversation_records.append({
                "round": 0,
                "agent": agent.role,
                "message": response
            })
        
        # Collaboration rounds
        for round_num in range(1, rounds + 1):
            # Each agent responds to the previous round
            for i, agent in enumerate(self.agents):
                # Get the other agent's response from previous round
                other_agent = self.agents[1 - i]  # For 2 agents
                other_response = initial_responses[1 - i]["response"] if round_num == 1 else conversation_records[-2]["message"]
                
                prompt = f"""
                Your teammate ({other_agent.role}) said: 
                
                "{other_response}"
                
                Based on your expertise as a {agent.role} and considering your teammate's perspective, 
                please refine your analysis or provide additional insights on {topic}.
                If you agree with certain points, acknowledge them.
                If you have different perspectives, explain your reasoning.
                Work toward developing an integrated viewpoint with your teammate.
                """
                
                response = agent.chat(prompt)
                conversation_records.append({
                    "round": round_num,
                    "agent": agent.role,
                    "message": response
                })
        
        # Final consensus
        consensus_prompt = f"""
        Review the entire discussion your team has had about {topic}.
        As a {self.agents[0].role}, collaborate with your teammate to provide a final consensus view that
        integrates both of your perspectives. This should represent your team's collective expertise and judgment.
        """
        
        consensus_response = self.agents[0].chat(consensus_prompt)
        conversation_records.append({
            "round": "consensus",
            "agent": f"{self.name} Team",
            "message": consensus_response
        })
        
        self.team_conversation = conversation_records
        return conversation_records
  

## LunarSurvivalSystem class

In [2]:
class LunarSurvivalSystem:
    """Main system for lunar survival challenge."""
    
    def __init__(self):
        """Initialize the lunar survival system with teams and meta agent."""
        
        # Define survival items
        self.survival_items = [
            {"name": "Oxygen Tanks", "description": "Two 100-pound tanks of oxygen"},
            {"name": "Water", "description": "20 liters of water"},
            {"name": "Stellar Map", "description": "Stellar map of the moon's constellation"},
            {"name": "Food Concentrate", "description": "Food concentrate for survival"},
            {"name": "Solar-Powered FM Receiver-Transmitter", "description": "Communication device"},
            {"name": "50 Feet of Nylon Rope", "description": "Strong utility rope"},
            {"name": "First Aid Kit", "description": "Contains medication, bandages"},
            {"name": "Parachute Silk", "description": "Large sheet of parachute silk"},
            {"name": "Life Raft", "description": "Self-inflating life raft"},
            {"name": "Signal Flares", "description": "Emergency signal flares"},
            {"name": "Pistols", "description": "Two .45 caliber pistols"},
            {"name": "Dehydrated Milk", "description": "One case of dehydrated milk"},
            {"name": "Portable Heating Unit", "description": "Portable heating unit"},
            {"name": "Magnetic Compass", "description": "Standard magnetic compass"},
            {"name": "Matches", "description": "Box of matches"}
        ]
        
        # Create knowledge acquisition team
        self.knowledge_team = Team(
            name="Knowledge Acquisition",
            goal="gather and analyze information about the lunar environment and human survival needs",
            members=[
                {
                    "role": "Science Analyst",
                    "expertise_description": "specializes in lunar physics, environmental conditions, and how items function in the lunar environment"
                },
                {
                    "role": "Resource Analyst",
                    "expertise_description": "specializes in human physiological needs, survival requirements, and resource management in extreme conditions"
                }
            ]
        )
        
        # Create technical reasoning team
        self.technical_team = Team(
            name="Technical Reasoning",
            goal="evaluate items based on scientific principles and established space procedures",
            members=[
                {
                    "role": "Science Reasoner",
                    "expertise_description": "applies scientific knowledge to evaluate how effectively items will function in the lunar environment"
                },
                {
                    "role": "Protocol Reasoner",
                    "expertise_description": "applies standard space agency procedures and protocols to evaluate items based on established guidelines"
                }
            ]
        )
        
        # Create practical reasoning team
        self.practical_team = Team(
            name="Practical Reasoning",
            goal="evaluate items based on survival needs and contingency planning",
            members=[
                {
                    "role": "Resource Reasoner",
                    "expertise_description": "optimizes resource allocation based on survival hierarchy and human needs"
                },
                {
                    "role": "Creative Reasoner",
                    "expertise_description": "considers edge cases, alternative item uses, and unexpected scenarios"
                }
            ]
        )
        
        # Create meta reasoning agent
        self.meta_agent = Agent(
            instruction="You are responsible for integrating multiple team perspectives and making final decisions.",
            role="Meta Reasoner",
            expertise_description="specializes in holistic reasoning, conflict resolution, and integrated decision making",
            model_info="gpt-4o"  # Use more capable model for meta reasoning
        )
        
        # Track system state
        self.knowledge_base = {}
        self.technical_ranking = []
        self.practical_ranking = []
        self.final_ranking = []
        self.justifications = {}
        
    #################################################################################
    
    def run(self) -> Dict[str, Any]:
        """
        Run the complete lunar survival challenge analysis.
        
        Returns:
            Dict containing final rankings and justifications
        """
        # Step 1: Knowledge Acquisition Phase
        self._knowledge_acquisition_phase()
        
        # Step 2: Technical Team Reasoning
        self._technical_reasoning_phase()
        
        # Step 3: Practical Team Reasoning
        self._practical_reasoning_phase()
        
        # Step 4: Cross-Team Integration
        self._cross_team_integration()
        
        # Step 5: Meta Reasoning and Final Ranking
        self._meta_reasoning_phase()
        
        # Prepare and return results
        results = {
            "final_ranking": self.final_ranking,
            "justifications": self.justifications,
            "knowledge_base": self.knowledge_base,
            "technical_ranking": self.technical_ranking,
            "practical_ranking": self.practical_ranking
        }
        
        return results
    
    #################################################################################
    

    def _knowledge_acquisition_phase(self):
        """
        Execute the knowledge acquisition phase where agents gather and integrate 
        knowledge about lunar environment and survival needs.
        """
        print("Starting Knowledge Acquisition Phase...")

        # Step 1: Gather lunar environmental information
        lunar_environment_prompt = """
        As a Science Analyst, your task is to gather comprehensive information about the lunar environment.
        Focus on:
        1. Physical conditions (vacuum, temperature extremes, radiation, gravity)
        2. Surface characteristics (terrain, dust, etc.)
        3. Day/night cycle and their implications
        4. How these conditions would affect the functioning of various items and human survival

        Provide a structured knowledge base that will be useful for evaluating survival items.
        """

        # Step 2: Gather human survival needs information
        survival_needs_prompt = """
        As a Resource Analyst, your task is to gather comprehensive information about human survival needs in the lunar environment.
        Focus on:
        1. Basic physiological needs (oxygen, water, food, temperature regulation)
        2. Timeframes for various survival needs (how long before critical failure)
        3. Special considerations for the lunar environment
        4. Hierarchy of survival priorities

        Provide a structured knowledge base that will be useful for evaluating survival items.
        """

        # Get responses from knowledge team
        env_response = self.knowledge_team.agents[0].chat(lunar_environment_prompt)
        needs_response = self.knowledge_team.agents[1].chat(survival_needs_prompt)

        # Step 3: Collaborative knowledge integration
        integration_prompt = f"""
        You need to collaborate to create an integrated knowledge base for lunar survival.

        The Science Analyst has provided information about the lunar environment:
        {env_response}

        The Resource Analyst has provided information about human survival needs:
        {needs_response}

        Work together to create an integrated knowledge framework that connects:
        1. Environmental challenges with human survival needs
        2. How different environmental factors affect survival priorities
        3. Critical timeframes and dependencies

        This integrated knowledge will be used to evaluate survival items, so make it practical and comprehensive.
        """

        # Collaborative knowledge integration through team discussion
        knowledge_integration = self.knowledge_team.collaborate(
            topic="Creating an integrated lunar survival knowledge base",
            rounds=2
        )

        # Extract the final consensus as our knowledge base
        integrated_knowledge = knowledge_integration[-1]["message"]

        # Store structured knowledge in our system
        self.knowledge_base = {
            "lunar_environment": env_response,
            "survival_needs": needs_response,
            "integrated_knowledge": integrated_knowledge
        }

        # Format knowledge for reasoning teams
        self._prepare_knowledge_for_reasoning_teams()

        print("Knowledge Acquisition Phase completed.")
        return self.knowledge_base
    
    #################################################################################
    

    def _prepare_knowledge_for_reasoning_teams(self):
        """
        Prepare knowledge in a format suitable for reasoning teams.
        This includes extracting key principles and facts from the knowledge base.
        """
        # Extract key environmental principles
        env_principles_prompt = f"""
        Based on this knowledge about the lunar environment:

        {self.knowledge_base['lunar_environment']}

        Please extract 10-15 key scientific principles that are most relevant for evaluating survival items.
        Format as a numbered list of concise statements that can be directly used by the reasoning teams.
        """
        env_principles = self.knowledge_team.agents[0].chat(env_principles_prompt)

        # Extract key survival principles
        survival_principles_prompt = f"""
        Based on this knowledge about human survival needs:

        {self.knowledge_base['survival_needs']}

        Please extract 10-15 key survival principles that are most relevant for evaluating survival items.
        Format as a numbered list of concise statements that can be directly used by the reasoning teams.
        Include time-critical factors and priority hierarchies.
        """
        survival_principles = self.knowledge_team.agents[1].chat(survival_principles_prompt)

        # Add to knowledge base
        self.knowledge_base["environmental_principles"] = env_principles
        self.knowledge_base["survival_principles"] = survival_principles

        # Create item-specific knowledge
        item_analysis_prompt = f"""
        Based on the lunar environment information and survival principles:

        Environmental principles:
        {env_principles}

        Survival principles:
        {survival_principles}

        For each of these survival items, provide a brief analysis of:
        1. How it would function in the lunar environment
        2. Its potential survival utility
        3. Any special considerations

        Be concise but thorough. This information will be used by reasoning teams to evaluate the items.

        Items to analyze:
        {", ".join([item["name"] for item in self.survival_items])}
        """

        # Collaborative item analysis
        item_analysis = self.knowledge_team.collaborate(
            topic="Analyzing survival items in the lunar context",
            rounds=1
        )

        # Extract and store the item analysis
        self.knowledge_base["item_analysis"] = item_analysis[-1]["message"]
    
    #################################################################################

    def _technical_reasoning_phase(self):
        """
        Execute the technical reasoning phase where the Science Reasoner and Protocol Reasoner
        collaborate to evaluate items based on scientific principles and space procedures.
        """
        print("Starting Technical Reasoning Phase...")

        # Prepare item list for evaluation
        item_descriptions = "\n".join([f"- {item['name']}: {item['description']}" for item in self.survival_items])

        # Create a collaborative reasoning prompt using knowledge from acquisition phase
        reasoning_prompt = f"""
        You are tasked with evaluating lunar survival items from a technical perspective.

        Environmental Knowledge:
        {self.knowledge_base['environmental_principles']}

        Item Analysis:
        {self.knowledge_base['item_analysis']}

        Items to evaluate:
        {item_descriptions}

        As a collaborative team:
        1. The Science Reasoner should focus on how each item functions in the lunar environment
        2. The Protocol Reasoner should focus on how each item aligns with standard space procedures
        3. Together, develop an integrated technical evaluation

        Your task is to collectively rank these items from most important (1) to least important (15) 
        for lunar survival based on your technical expertise.

        For each item, provide:
        1. Its rank
        2. A brief technical justification focusing on scientific principles and protocols
        3. A confidence score (1-10)

        Work together to produce an integrated technical ranking.
        """

        # Facilitate collaborative reasoning in the technical team
        technical_collaboration = self.technical_team.collaborate(
            topic="Technical evaluation and ranking of lunar survival items",
            rounds=3  # More rounds for thorough discussion
        )

        # Extract the consensus ranking
        technical_consensus = technical_collaboration[-1]["message"]

        # Parse the ranking from the consensus message
        # This is a simplified parsing approach - in a production system, you'd want more robust parsing
        self.technical_ranking = self._parse_ranking(technical_consensus)

        print("Technical Reasoning Phase completed.")
        return self.technical_ranking
    
    #################################################################################
    
    def _practical_reasoning_phase(self):
        """
        Execute the practical reasoning phase where the Resource Reasoner and Creative Reasoner
        collaborate to evaluate items based on survival needs and contingency planning.
        """
        print("Starting Practical Reasoning Phase...")

        # Prepare item list for evaluation
        item_descriptions = "\n".join([f"- {item['name']}: {item['description']}" for item in self.survival_items])

        # Create a collaborative reasoning prompt using knowledge from acquisition phase
        reasoning_prompt = f"""
        You are tasked with evaluating lunar survival items from a practical perspective.

        Survival Knowledge:
        {self.knowledge_base['survival_principles']}

        Item Analysis:
        {self.knowledge_base['item_analysis']}

        Items to evaluate:
        {item_descriptions}

        As a collaborative team:
        1. The Resource Reasoner should focus on how each item addresses critical survival needs
        2. The Creative Reasoner should focus on alternative uses and contingency scenarios
        3. Together, develop an integrated practical evaluation

        Your task is to collectively rank these items from most important (1) to least important (15) 
        for lunar survival based on your practical expertise.

        For each item, provide:
        1. Its rank
        2. A brief practical justification focusing on survival utility and adaptability
        3. A confidence score (1-10)

        Work together to produce an integrated practical ranking.
        """

        # Facilitate collaborative reasoning in the practical team
        practical_collaboration = self.practical_team.collaborate(
            topic="Practical evaluation and ranking of lunar survival items",
            rounds=3  # More rounds for thorough discussion
        )

        # Extract the consensus ranking
        practical_consensus = practical_collaboration[-1]["message"]

        # Parse the ranking from the consensus message
        self.practical_ranking = self._parse_ranking(practical_consensus)

        print("Practical Reasoning Phase completed.")
        return self.practical_ranking
    
    #################################################################################
    

    def _parse_ranking(self, consensus_text):
        """
        Parse ranking information from consensus text.

        Args:
            consensus_text: Text containing ranking information

        Returns:
            List of dictionaries with ranking information
        """
        # This is a simplified parser
        # In a production system, you'd want to use more robust parsing with NLP

        # Initialize list to store parsed rankings
        parsed_rankings = []

        # Try to extract items with rankings
        lines = consensus_text.split("\n")
        current_item = {}

        for line in lines:
            line = line.strip()

            # Look for rank pattern (number followed by period or bracket)
            if line and (line[0].isdigit() and (len(line) > 1 and (line[1] == '.' or line[1] == ')'))):
                # If we were processing an item, save it before starting a new one
                if current_item and 'name' in current_item:
                    parsed_rankings.append(current_item)

                # Start a new item
                parts = line.split(" ", 1)
                if len(parts) > 1:
                    rank_str = parts[0].rstrip('.):')
                    item_name = parts[1].strip().split(":", 1)[0].strip()
                    current_item = {
                        "rank": int(rank_str),
                        "name": item_name,
                        "justification": "",
                        "confidence": 0
                    }

            # Add to justification if we're processing an item
            elif current_item and 'name' in current_item:
                # Look for confidence pattern
                if "confidence:" in line.lower() or "confidence score:" in line.lower():
                    try:
                        # Extract confidence value
                        conf_parts = line.split(":", 1)
                        if len(conf_parts) > 1:
                            conf_str = conf_parts[1].strip().split("/")[0].strip()
                            current_item["confidence"] = float(conf_str)
                    except:
                        pass  # If parsing fails, keep default confidence
                else:
                    current_item["justification"] += line + " "

        # Add the last item if exists
        if current_item and 'name' in current_item:
            parsed_rankings.append(current_item)

        # Sort by rank
        parsed_rankings.sort(key=lambda x: x["rank"])

        return parsed_rankings
    
    #################################################################################
    

    def _cross_team_integration(self):
        """
        Execute the cross-team integration phase where both teams reconcile their rankings
        and identify areas of agreement and disagreement.
        """
        print("Starting Cross-Team Integration Phase...")

        # Compare rankings to identify agreements and disagreements
        agreement_items, disagreement_items = self._identify_agreements_disagreements()

        # Format the findings for both teams
        technical_ranking_formatted = self._format_ranking(self.technical_ranking)
        practical_ranking_formatted = self._format_ranking(self.practical_ranking)

        disagreements_formatted = ""
        for item in disagreement_items:
            tech_rank = next((i["rank"] for i in self.technical_ranking if i["name"] == item), "Not ranked")
            prac_rank = next((i["rank"] for i in self.practical_ranking if i["name"] == item), "Not ranked")
            disagreements_formatted += f"- {item}: Technical rank {tech_rank}, Practical rank {prac_rank}\n"

        # Create integration prompt
        integration_prompt = f"""
        Both the Technical and Practical teams have evaluated the lunar survival items.

        Technical Team Ranking:
        {technical_ranking_formatted}

        Practical Team Ranking:
        {practical_ranking_formatted}

        There is significant disagreement on these items:
        {disagreements_formatted}

        For each item of disagreement, please:
        1. Have both teams explain their reasoning
        2. Discuss the core factors behind the different evaluations
        3. Try to find common ground or explicit reasons for maintaining different views

        The goal is not to force agreement, but to understand each other's perspectives 
        and provide clear reasoning for the Meta Reasoner to consider.
        """

        # Create a temporary team for cross-team integration
        # This includes one agent from each team to represent their team's perspective
        integration_team = Team(
            name="Integration",
            goal="reconcile technical and practical perspectives on lunar survival items",
            members=[
                {
                    "role": "Technical Representative",
                    "expertise_description": "represents the technical team's perspective on item evaluation"
                },
                {
                    "role": "Practical Representative",
                    "expertise_description": "represents the practical team's perspective on item evaluation"
                }
            ]
        )

        # Facilitate cross-team discussion
        integration_discussion = integration_team.collaborate(
            topic="Reconciling technical and practical perspectives on lunar survival items",
            rounds=3  # More rounds for thorough discussion of disagreements
        )

        # Store the integration discussion
        self.integration_discussion = integration_discussion

        print("Cross-Team Integration Phase completed.")
        return self.integration_discussion
    
    
    #################################################################################

    def _identify_agreements_disagreements(self):
        """
        Identify items with agreement and disagreement between technical and practical rankings.

        Returns:
            Tuple of (agreement_items, disagreement_items)
        """
        agreement_items = []
        disagreement_items = []

        # Convert rankings to dictionaries for easier comparison
        tech_ranks = {item["name"]: item["rank"] for item in self.technical_ranking}
        prac_ranks = {item["name"]: item["rank"] for item in self.practical_ranking}

        # Check all items
        for item in self.survival_items:
            name = item["name"]

            # Check if item is in both rankings
            if name in tech_ranks and name in prac_ranks:
                # Check if ranks are similar (within 2 positions)
                if abs(tech_ranks[name] - prac_ranks[name]) <= 2:
                    agreement_items.append(name)
                else:
                    disagreement_items.append(name)
            else:
                # If item is missing from either ranking, consider it a disagreement
                disagreement_items.append(name)

        return agreement_items, disagreement_items
    
    
    #################################################################################

    def _format_ranking(self, ranking):
        """Format ranking for display in prompts."""
        formatted = ""
        for item in ranking:
            formatted += f"{item['rank']}. {item['name']}: {item['justification'].strip()} (Confidence: {item['confidence']})\n\n"
        return formatted


    
    #################################################################################
    
    
    def _meta_reasoning_phase(self):
        """
        Execute the meta reasoning phase where the Meta Reasoner integrates perspectives
        and makes final decisions.
        """
        print("Starting Meta Reasoning Phase...")

        # Extract the final integration discussion
        integration_consensus = self.integration_discussion[-1]["message"]

        # Format rankings and discussions for meta reasoning
        technical_ranking_formatted = self._format_ranking(self.technical_ranking)
        practical_ranking_formatted = self._format_ranking(self.practical_ranking)

        # Create meta reasoning prompt
        meta_prompt = f"""
        As the Meta Reasoner, your task is to integrate the technical and practical perspectives
        on lunar survival items and make final decisions.

        Knowledge Base:
        {self.knowledge_base['integrated_knowledge'][:1000]}... (abbreviated)

        Technical Team Ranking:
        {technical_ranking_formatted}

        Practical Team Ranking:
        {practical_ranking_formatted}

        Integration Discussion:
        {integration_consensus}

        Your task:
        1. Create a final ranking of all 15 items from most important (1) to least important (15)
        2. For each item, provide a comprehensive justification that integrates both technical and practical perspectives
        3. Address any significant disagreements and explain your reasoning for the final position
        4. Assign a final confidence score (1-10) for each item

        The final ranking should represent a holistic evaluation that considers all perspectives.
        """

        # Get meta reasoning response
        meta_response = self.meta_agent.chat(meta_prompt)

        # Parse final ranking
        self.final_ranking = self._parse_ranking(meta_response)

        # Generate comprehensive justifications
        self._generate_comprehensive_justifications()

        print("Meta Reasoning Phase completed.")
        return self.final_ranking
    
    #################################################################################
    

    def _generate_comprehensive_justifications(self):
        """
        Generate comprehensive justifications for each item in the final ranking.
        These justifications integrate technical and practical perspectives and
        provide a complete rationale for the item's position.
        """
        # For each item in the final ranking
        for item_ranking in self.final_ranking:
            item_name = item_ranking["name"]

            # Get technical and practical perspectives
            tech_perspective = next((item for item in self.technical_ranking if item["name"] == item_name), None)
            prac_perspective = next((item for item in self.practical_ranking if item["name"] == item_name), None)

            # Create justification prompt
            justification_prompt = f"""
            You need to create a comprehensive justification for {item_name} which is ranked #{item_ranking['rank']} in the final lunar survival items ranking.

            Technical Perspective:
            {tech_perspective['justification'] if tech_perspective else 'Not provided'}

            Practical Perspective:
            {prac_perspective['justification'] if prac_perspective else 'Not provided'}

            Your current justification:
            {item_ranking['justification']}

            Please integrate these perspectives into a comprehensive, well-structured justification that explains:
            1. The item's function in the lunar environment
            2. Its survival utility and critical timeframes
            3. Its relation to established protocols
            4. Alternative uses and contingency value
            5. Why it deserves its specific ranking position

            This should be thorough yet concise, with clear reasoning.
            """

            # Get comprehensive justification
            comprehensive_justification = self.meta_agent.chat(justification_prompt)

            # Store the comprehensive justification
            self.justifications[item_name] = {
                "rank": item_ranking["rank"],
                "technical_perspective": tech_perspective["justification"] if tech_perspective else "Not provided",
                "practical_perspective": prac_perspective["justification"] if prac_perspective else "Not provided",
                "final_justification": comprehensive_justification,
                "confidence": item_ranking["confidence"]
            }

        return self.justifications

    
    #################################################################################
    

    def generate_report(self):
        """
        Generate a comprehensive final report on the lunar survival item rankings.

        Returns:
            A formatted report string
        """
        # Ensure we have final rankings
        if not self.final_ranking:
            raise ValueError("No final ranking available. Run the system first.")

        # Sort items by rank
        sorted_items = sorted(self.final_ranking, key=lambda x: x["rank"])

        # Build report header
        report = """
        # Lunar Survival Challenge - Final Report

        ## Executive Summary

        This report presents the final rankings for the lunar survival challenge items based on a collaborative
        multi-agent reasoning system. The system integrated technical perspectives (focusing on scientific principles
        and space procedures) with practical perspectives (focusing on survival needs and contingency planning).

        ## Methodology

        The evaluation followed a structured collaborative process:

        1. **Knowledge Acquisition Phase**: Science and Resource analysts gathered environmental and survival knowledge
        2. **Technical Reasoning Phase**: Science and Protocol reasoners evaluated items from a technical perspective
        3. **Practical Reasoning Phase**: Resource and Creative reasoners evaluated items from a practical perspective
        4. **Cross-Team Integration**: Team representatives reconciled perspectives and addressed disagreements
        5. **Meta Reasoning**: A holistic analysis integrated all perspectives into final rankings

        ## Key Findings

        The most critical items for lunar survival are those addressing immediate physiological needs and
        those enabling communication and navigation to the rendezvous point.

        ## Final Rankings

        """

        # Add each item with its rank and justification
        for item in sorted_items:
            report += f"""
        ### {item['rank']}. {item['name']} (Confidence: {item['confidence']}/10)

        {self.justifications.get(item['name'], {}).get('final_justification', item['justification'])}

        """

        # Add appendices with team perspectives
        report += """
        ## Appendix A: Technical Perspective

        The Technical team (Science and Protocol Reasoners) evaluated items based on lunar environmental 
        factors and standard space procedures.

        """

        for item in sorted(self.technical_ranking, key=lambda x: x["rank"]):
            report += f"""
        **{item['rank']}. {item['name']}**: {item['justification']}

        """

        report += """
        ## Appendix B: Practical Perspective

        The Practical team (Resource and Creative Reasoners) evaluated items based on survival needs
        and contingency planning.

        """

        for item in sorted(self.practical_ranking, key=lambda x: x["rank"]):
            report += f"""
        **{item['rank']}. {item['name']}**: {item['justification']}

        """

        return report
    
    
    #################################################################################

    def save_report(self, filename="lunar_survival_report.md"):
        """
        Save the final report to a file.

        Args:
            filename: Name of file to save report to
        """
        report = self.generate_report()

        with open(filename, "w") as f:
            f.write(report)

        print(f"Report saved to {filename}")

## Main Execution function

In [ ]:
"""
Lunar Survival Challenge - Multi-Agent Collaborative System
"""

import os
import time
import json
from dotenv import load_dotenv


"""Main execution function"""

# Load environment variables
load_dotenv()

# Check for API key
if not os.environ.get('OPENAI_API_KEY'):
    print("Error: OPENAI_API_KEY environment variable not set.")
    print("Please set your OpenAI API key in a .env file or environment variable.")
    return

print("=== Lunar Survival Challenge - Multi-Agent System ===")
print("Initializing system...")

# Create the lunar survival system
lunar_system = LunarSurvivalSystem()

# Track timing
start_time = time.time()

# Run the complete analysis
print("\nRunning complete lunar survival analysis...")
results = lunar_system.run()

# Generate and save the report
print("\nGenerating final report...")
lunar_system.save_report()

# Save raw results
with open("lunar_survival_results.json", "w") as f:
    # Convert complex objects to serializable format
    serializable_results = {
        "final_ranking": results["final_ranking"],
        "technical_ranking": results["technical_ranking"],
        "practical_ranking": results["practical_ranking"],
        # Exclude or simplify other complex objects
    }
    json.dump(serializable_results, f, indent=2)

# Print final rankings
print("\n=== Final Rankings ===")
for item in sorted(results["final_ranking"], key=lambda x: x["rank"]):
    print(f"{item['rank']}. {item['name']} (Confidence: {item['confidence']}/10)")

# Print timing information
end_time = time.time()
print(f"\nAnalysis completed in {end_time - start_time:.2f} seconds.")
print(f"Final report saved to lunar_survival_report.md")
print(f"Raw results saved to lunar_survival_results.json")